<a href="https://colab.research.google.com/github/01Joseph-Hwang10/AI_fundamentals/blob/main/aip2_assign2_20211381.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
from sklearn.linear_model import LinearRegression
import numpy as np
X = np.array([[5.0,3.4,1.5,0.2],[6.1,2.8,4.7,1.2],[6.9,3.1,5.1,2.3],[5.8,2.7,4.1,1.0]])
y = np.array([[2],[1],[3],[1]])

lin_reg = LinearRegression(fit_intercept=False)
lin_reg.fit(X, y)

print(lin_reg.coef_)
print(lin_reg.intercept_)

[[ 0.04312268  0.75390335 -0.76728625  1.86022305]]
0.0


In [13]:
import numpy as np
import pandas as pd

housing = pd.read_csv('https://raw.githubusercontent.com/ageron/handson-ml/master/datasets/housing/housing.csv')
housing.head(10)

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY
5,-122.25,37.85,52.0,919.0,213.0,413.0,193.0,4.0368,269700.0,NEAR BAY
6,-122.25,37.84,52.0,2535.0,489.0,1094.0,514.0,3.6591,299200.0,NEAR BAY
7,-122.25,37.84,52.0,3104.0,687.0,1157.0,647.0,3.1200,241400.0,NEAR BAY
8,-122.26,37.84,42.0,2555.0,665.0,1206.0,595.0,2.0804,226700.0,NEAR BAY
9,-122.25,37.84,52.0,3549.0,707.0,1551.0,714.0,3.6912,261100.0,NEAR BAY


In [14]:
median = housing['total_bedrooms'].median()
housing['total_bedrooms'] = housing['total_bedrooms'].fillna(median)

In [15]:
housing['rooms_per_household'] = housing['total_rooms'] / housing['households']
# del housing['total_rooms']

In [16]:
# housing['rooms_per_household'] = housing['total_rooms'] / housing['households'] # Feature를 이걸 쓰는거랑

housing['bedrooms_per_household'] = housing['total_bedrooms'] / housing['total_rooms'] # Feature를 이걸 쓰는거랑 결과가 조금씩 다릅니다.

del housing['total_rooms'], housing['total_bedrooms']

In [17]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

col_list = list(housing)
col_list.remove("ocean_proximity") # text type
col_list.remove("median_house_value") # target variables needs not to be scaled

# generate a new dataframe that consist of numeric type only
housing_numeric = housing[col_list]
housing_scaled = scaler.fit_transform(housing_numeric)
# Data type conversion from 'Series' to 'DataFrame'
housing_scaled_df = pd.DataFrame(housing_scaled, index=housing_numeric.index, columns=housing_numeric.columns)

# Concatenate
housing = pd.concat([housing_scaled_df, housing["median_house_value"], housing["ocean_proximity"]], axis=1)
housing.head(10)

,longitude,latitude,housing_median_age,population,households,median_income,rooms_per_household,bedrooms_per_household,median_house_value,ocean_proximity
0,-1.327835,1.052548,0.982143,-0.974429,-0.977033,2.344766,0.628559,-1.029988,452600.0,NEAR BAY
1,-1.322844,1.043185,-0.607019,0.861439,1.669961,2.332238,0.327041,-0.888897,358500.0,NEAR BAY
2,-1.332827,1.038503,1.856182,-0.820777,-0.843637,1.782699,1.155620,-1.291686,352100.0,NEAR BAY
3,-1.337818,1.038503,1.856182,-0.766028,-0.733781,0.932968,0.156966,-0.449613,341300.0,NEAR BAY
4,-1.337818,1.038503,1.856182,-0.759847,-0.629157,-0.012881,0.344711,-0.639087,342200.0,NEAR BAY
5,-1.337818,1.038503,1.856182,-0.894071,-0.801787,0.087447,-0.269730,0.275563,269700.0,NEAR BAY
6,-1.337818,1.033821,1.856182,-0.292712,0.037823,-0.111366,-0.200918,-0.320242,299200.0,NEAR BAY
7,-1.337818,1.033821,1.856182,-0.237079,0.385698,-0.395137,-0.255232,0.115458,241400.0,NEAR BAY
8,-1.342809,1.033821,1.061601,-0.193810,0.249687,-0.942359,-0.458703,0.712372,226700.0,NEAR BAY
9,-1.337818,1.033821,1.856182,0.110844,0.560944,-0.094470,-0.185283,-0.223507,261100.0,NEAR BAY


In [18]:
# One-hot encoding
housing = pd.get_dummies(housing)
housing.head(10)

,longitude,latitude,housing_median_age,population,households,median_income,rooms_per_household,bedrooms_per_household,median_house_value,ocean_proximity_<1H OCEAN,ocean_proximity_INLAND,ocean_proximity_ISLAND,ocean_proximity_NEAR BAY,ocean_proximity_NEAR OCEAN
0,-1.327835,1.052548,0.982143,-0.974429,-0.977033,2.344766,0.628559,-1.029988,452600.0,0,0,0,1,0
1,-1.322844,1.043185,-0.607019,0.861439,1.669961,2.332238,0.327041,-0.888897,358500.0,0,0,0,1,0
2,-1.332827,1.038503,1.856182,-0.820777,-0.843637,1.782699,1.155620,-1.291686,352100.0,0,0,0,1,0
3,-1.337818,1.038503,1.856182,-0.766028,-0.733781,0.932968,0.156966,-0.449613,341300.0,0,0,0,1,0
4,-1.337818,1.038503,1.856182,-0.759847,-0.629157,-0.012881,0.344711,-0.639087,342200.0,0,0,0,1,0
5,-1.337818,1.038503,1.856182,-0.894071,-0.801787,0.087447,-0.269730,0.275563,269700.0,0,0,0,1,0
6,-1.337818,1.033821,1.856182,-0.292712,0.037823,-0.111366,-0.200918,-0.320242,299200.0,0,0,0,1,0
7,-1.337818,1.033821,1.856182,-0.237079,0.385698,-0.395137,-0.255232,0.115458,241400.0,0,0,0,1,0
8,-1.342809,1.033821,1.061601,-0.193810,0.249687,-0.942359,-0.458703,0.712372,226700.0,0,0,0,1,0
9,-1.337818,1.033821,1.856182,0.110844,0.560944,-0.094470,-0.185283,-0.223507,261100.0,0,0,0,1,0


In [19]:
# training - test seperation
from sklearn.model_selection import train_test_split

train_set, test_set = train_test_split(housing, test_size=0.2, random_state=45)

print('# of train_set : %.0f, # of test_set : %.0f' % (train_set.shape[0], test_set.shape[0]))

# of train_set : 16512, # of test_set : 4128


In [20]:
# feature and label seperation of training set
train_set_features = train_set.drop('median_house_value', axis=1)
train_set_target = train_set['median_house_value'].copy()

In [22]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

lin_reg = LinearRegression()
lin_reg.fit(train_set_features, train_set_target)

print('Learned Parameters : ')
print('Coefficients: ', lin_reg.coef_)
print('bias: ', lin_reg.intercept_)

Learned Parameters : 
Coefficients:  [-55774.14086907 -56964.69903623  12971.17103303 -53562.80929239
  57272.02939275  74147.51210095   6564.46012412  10459.02523705
 -25921.29448802 -63911.36887214 143057.05886283 -30846.20870803
 -22378.18679464]
bias:  244824.9222886056


In [24]:
test_set_features = test_set.drop('median_house_value', axis=1)
test_set_target = test_set['median_house_value'].copy()

final_model = lin_reg
final_predictions = final_model.predict(test_set_features)

print("Test_r2_score: ", r2_score(test_set_target, final_predictions))

delta = test_set_target - final_predictions
print('Errors (%) in housing value prediction : ', np.mean(np.abs(delta) / test_set_target))

# RMSE
final_mse = mean_squared_error(test_set_target, final_predictions)
final_rmse = np.sqrt(final_mse)

Test_r2_score:  0.6308608545588804
Errors (%) in housing value prediction :  0.2861199789930685


In [25]:
print(final_rmse)

69987.25423203211
